<a href="https://colab.research.google.com/github/jcims123/spark_in_colab/blob/main/20250802_spark_colab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===== CORRECTED PYSPARK SETUP FOR GOOGLE COLAB =====

# Step 1: Install Java (required for Spark)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Step 2: Download Apache Spark 3.5.0 (stable version that works reliably)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

# Step 3: Install findspark
!pip install findspark

# Step 4: Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

# Step 5: Initialize Spark
import findspark
findspark.init()

print("✅ PySpark 3.5.0 installed successfully!")

✅ PySpark 3.5.0 installed successfully!
